# Analisis de bases de datos 

In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import datetime as dt
import warnings
import matplotlib.pyplot as plt


warnings.filterwarnings("ignore")

In [2]:
ticker_name = 'BC'

data = yf.download(ticker_name, start='2000-01-01', end='2020-01-01')
print(data.shape)
data.head()

[*********************100%***********************]  1 of 1 completed
(5031, 6)


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-01-03,22.0000,22.09375,21.6875,21.9375,15.495515,286700
2000-01-04,21.9375,22.12500,20.8750,20.9375,14.789170,338700
2000-01-05,20.6875,21.62500,20.6875,21.0625,14.877457,214000
2000-01-06,21.1250,21.31250,20.8750,21.2500,15.009907,232700
2000-01-07,21.2500,21.62500,20.9375,21.3750,15.098194,207800


In [3]:
data.shape

(5031, 6)

In [4]:
df = pd.DataFrame()
df["Date"] = pd.to_datetime(data.index).date
df["Close"] = data["Close"].values
df.head()

,Date,Close
0,2000-01-03,21.9375
1,2000-01-04,20.9375
2,2000-01-05,21.0625
3,2000-01-06,21.2500
4,2000-01-07,21.3750


In [5]:
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure()
# Suponiendo que ya tienes el DataFrame df

fig.add_trace(go.Scatter(
    x=df["Date"],
    y=df["Close"],
    name="NeuralProphet Forescasting",
    line=dict(color='blue', width=2)
))
fig.update_xaxes(type='category')  # Para que las fechas se muestren correctamente

# Centrar el título
fig.update_layout(title_text='Precio de acciones Bancolombia', title_x=0.5)
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1d",
                     step="day",
                     stepmode="backward"),
                dict(count=7,
                     label="1w",
                     step="day",
                     stepmode="backward"),
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=3,
                     label="3m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    ),
    yaxis=dict(
        autorange=True,
        type="linear"
    )
)

fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Price",
    legend=dict(
        orientation="v",
        yanchor="top",
        y=1,
        xanchor="left",
        x=-.35
    )
)
fig.show()
